In [14]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)

In [17]:
embeddings = load_embeddings(config, tok2id)

INFO:Initialized embeddings.


In [18]:
embeddings.shape

(10008, 50)

In [29]:
from tf_linear_classifier import TfLinearClassifier

In [30]:
model = TfLinearClassifier(embedding=embeddings)

In [31]:
model.fit_tfdata(tfConfig, batches_to_eval=50, max_iter=10)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...
INFO:took 0 s
INFO:Starting epoch 1
INFO: == Evaluating on development data after batch 50
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.607, Recall: 0.607, F1: 0.607
INFO: == Evaluating on development data after batch 100
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.629, Recall: 0.629, F1: 0.629
INFO: == Evaluating on development data after batch 150
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.660, Recall: 0.660, F1: 0.660
INFO: == Evaluating on development data after batch 200
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.664, Recall: 0.664, F1: 0.664
INFO: == Evaluating on development data after batch 250
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.697, Recall: 0.697, F1: 0.697
INFO: == Evaluating on development data after bat

INFO: == Evaluating on development data after batch 400
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.952, Recall: 0.952, F1: 0.952
INFO: == Evaluating on development data after batch 450
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.940, Recall: 0.940, F1: 0.940
INFO: == Evaluating on development data after batch 500
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.952, Recall: 0.952, F1: 0.952
INFO: == Evaluating on development data after batch 550
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.945, Recall: 0.945, F1: 0.945
INFO: == Evaluating on development data after batch 600
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.957, Recall: 0.957, F1: 0.957
INFO: == Evaluating on development data after batch 650
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.948, Recall: 0.9

INFO:== Precision: 0.977, Recall: 0.977, F1: 0.977
INFO: == Evaluating on development data after batch 800
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.968, Recall: 0.968, F1: 0.968
INFO: == Evaluating on development data after batch 850
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.970, Recall: 0.970, F1: 0.970
INFO: == Evaluating on development data after batch 900
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.970, Recall: 0.970, F1: 0.970
INFO: == Evaluating on development data after batch 950
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.968, Recall: 0.968, F1: 0.968
INFO: == Evaluating on development data after batch 1000
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.975, Recall: 0.975, F1: 0.975
INFO: = batch 1000
INFO:Finished epoch 6, ran 1000 batches
INFO:Starting epoch 6
INFO: == Evaluating on

INFO: == Evaluating on development data after batch 150
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.973, Recall: 0.973, F1: 0.973
INFO: == Evaluating on development data after batch 200
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.974, Recall: 0.974, F1: 0.974
INFO: == Evaluating on development data after batch 250
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.973, Recall: 0.973, F1: 0.973
INFO: == Evaluating on development data after batch 300
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.972, Recall: 0.972, F1: 0.972
INFO: == Evaluating on development data after batch 350
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.975, Recall: 0.975, F1: 0.975
INFO: == Evaluating on development data after batch 400
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.973, Recall: 0.9

INFO:== Precision: 0.982, Recall: 0.982, F1: 0.982
INFO: == Evaluating on development data after batch 550
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.978, Recall: 0.978, F1: 0.978
INFO: == Evaluating on development data after batch 600
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.978, Recall: 0.978, F1: 0.978
INFO: == Evaluating on development data after batch 650
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.979, Recall: 0.979, F1: 0.979
INFO: == Evaluating on development data after batch 700
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.979, Recall: 0.979, F1: 0.979
INFO: == Evaluating on development data after batch 750
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.983, Recall: 0.983, F1: 0.983
INFO: == Evaluating on development data after batch 800
INFO:Ran 40 batches
INFO:== Results for 40 batch

In [ ]:
l, p = model.evaluate_tfdata()